In [1]:
import psycopg2
import dateutil.parser
import os 
import sys
import itertools
import matplotlib.pyplot as plt
import database_connection as db_connection
import json
import pandas as pd
import numpy as np
from functools import lru_cache

ok
<connection object at 0x10f3706d0; dsn: 'user=postgres password=xxx dbname=database_asc host=localhost port=5432', closed: 0>


In [2]:
request  = """SELECT * FROM partner_alma_data.payload_pending_stat
ORDER BY equipment_id ASC"""
df = db_connection.send_request((request))
field='c28'

In [3]:
values_payload_eqp = {element:list() for element in df.equipment_id.unique() }

In [4]:
df_value_equipment_id = pd.DataFrame(df[['equipment_id','value']])

In [ ]:
def get_all_fields(df):
    all_fields = set()
    for val in df.value:
        try:
            all_fields = all_fields.union(set(list(json.loads(val).keys())))
        except:
            pass
    return all_fields
all_fields=get_all_fields(df_value_equipment_id)

In [ ]:
def get_sensors_value_eqp_datetime(field):
    
    values_payload_eqp = {element:list() for element in df.equipment_id.unique() }
    for eqp_id in values_payload_eqp.keys():
        temp = df_value_equipment_id.groupby('equipment_id').get_group(eqp_id)
        try:

            payloads = [(json.loads(element).get(field),json.loads(element).get("t")) for element in temp.value.values if json.loads(element).get(field)]
            values_payload_eqp[eqp_id]= payloads
        except:
            pass
    return values_payload_eqp        

In [ ]:
def get_by_day(list_tuple):
    groups = []
    uniquekeys = []
    for k, g in itertools.groupby(list_tuple, key=lambda x: x[1].date()):
       groups.append(k)    
       uniquekeys.append(list(g))

    return uniquekeys


In [ ]:
def get_graph(x,y,title):
    plt.plot(x,y)
    plt.title(title)
    plt.xticks(rotation=45) 
    plt.figure()

In [ ]:
def get_consecutive_date(list_date):
    res = list()
    for element in list_date:
        if len(res)==0:
            res.append(element)
        else:
            if abs((res[-1][-1][1]-element[0][1]).days)==1:
                res[-1]=unique_consecutive(res[-1],element)
            else:
                res.append(element)
    return res

In [ ]:
def get_graph_by_day(list_tuple):
    to_plot = get_by_day(list_tuple)
    
    for element in to_plot:
        x = [e[1] for e in element ]
        y = [e[0] for e in element]
        title = str(element[0][1].date())
        get_graph(x,y,title)
        

In [ ]:
def unique_consecutive(list1,list2):
    res = list()
    for element in list1:
        res.append(element)
    for element in list2:
        res.append(element)
    return res

In [ ]:
result = get_sensors_value_eqp_datetime(field)

In [ ]:
result

In [ ]:
def get_maxs(offset):
    if offset<=0:
        return
    if offset==1:
        max_temp = sorted([(len(result[key]),key) for key in result.keys()])[-1]
        eqpmax_date = [(element[0],dateutil.parser.parse(element[1])) for element in result[max_temp[-1]]]
        eqpmax_date=sorted(eqpmax_date,key = lambda x:x[1])
        return max_temp[1],eqpmax_date
    else:
        
        max_temp = sorted([(len(result[key]),key) for key in result.keys()])
        max_temp.reverse()
        keys =[ e[1] for e in max_temp[:offset]]
        max_temp = [ e[1] for e in max_temp[:offset]]
        eqpmaxs_dates = list()
        for value in max_temp: ##ici on a la list des id des n equipement ayant remontés le plus de données
            temp = ([(element[0],dateutil.parser.parse(element[1])) for element in result[value]])
            temp = sorted(temp,key = lambda x:x[1])
            eqpmaxs_dates.append(temp)
        return dict(zip(keys,eqpmaxs_dates))

In [ ]:
def optimized_offset(dict_values):
    return len([a for a in dict_values.items() if len(a[1])>0])

In [ ]:
#result -> len =304
# non vide = 28
#vide = 276
#\/
optimized_offset(result)

### print the group of consecutives datetime with values associated ### 
dataframes_per_consecutives_days

In [ ]:
dfs = list()
max_ = get_maxs(optimized_offset(result))
for key,value in max_.items(): 
    for element in get_by_day(value):
        temp = [(key,a,b) for a,b in element ]
        dfs.append(pd.DataFrame(data=temp, columns = ['Equipment_id','Value','Date']))
    for d in dfs:
        d['Value']=d['Value'].abs()

#for element in dataframes_per_consecutives_days:
#plt.title(element.Equipment_id.unique())
#plt.plot(element.Date,element.Value)
#plt.xticks(rotation=60)
#plt.show()

In [ ]:
dfs

In [ ]:
#eqp qui remontent des T°
s = set()
for element in dfs:
    s.add(element.Equipment_id.iloc[1])
print(list(s))

## ça semble ok 

In [ ]:
def save_datasets():
    path = os.getcwd()+"/DataFrames/"+str(field)
    choice = input('save ?')
    if(choice in ['yes',',no'] and choice =='yes'):
        for e in dfs:
            key = str(e.Equipment_id.unique()[-1])
            if not os.path.exists(path+"/"+key):
                os.mkdir(path+"/"+key)
                print("Directory " , path+"/"+key ,  " Created ")
            timestamp = str(e.Date[0].date())
            e.to_csv(path_or_buf=path+"/{}/{}.csv".format(key,timestamp))
            print('Dataset saved to '+path+'/'+key)
            #pour la suite on va sur colab -> diminution des temps de traitement et peut etre ML pour la modelisation de la courbe

#### save_datasets()